### Install Python SDK for Germini





In [ ]:
!pip install -U -q google-generativeai # Install the Python SDK

#Define environments and thoughts

In [ ]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


#text = generate_thought("What is the capital city of Japan?")
#expected answer: Tokyo
def generate_thought(prompt):
  print("Prompt:")
  print(prompt) #comment this line if you would not like to print prompt
  response = model.generate_content(prompt,
    generation_config=genai.types.GenerationConfig(
        candidate_count=1,
        max_output_tokens=500,
        temperature=0.0
    ))
  print("Response:")
  return response.text

def rule_thought(fact,rule,issue,hypo=""):
  add_prompt = ""
  prompt = "Considering the following facts:\n"+fact+"\n"
  prompt += "Considering the following rules:\n"+rule+"\n"
  if(hypo != ""):
    prompt += "Assuming these hypotheses are true in this case:\n"+hypo+"\n"
  if(hypo != ""):
    add_prompt = "and hypotheses"
  #prompt += "According to the given rules "+add_prompt+", please give yes/no answer whether "+issue+" (no breakdown)."
  prompt += "According to the given rules "+add_prompt+", please evaluate whether "+issue+"."
  return generate_thought(prompt)

def case_thought(precedent,outcome):
  prompt = "Considering the following case: \n"+precedent+"\n"
  prompt += "The case has the following outcome: \n"+outcome+"\n"
  prompt += "Please answer why the case has the outcome only in the following form: \n"
  prompt += "A person is typically guilty of theft if the following conditions hold: (1)... (2)... or "
  prompt += "A person is typically not guilty of theft if the following conditions hold: (1)... (2)..."
  return generate_thought(prompt)

def hypo_thought(fact,outcome):
  prompt = "Considering the following facts: \n"+fact+"\n"
  prompt += "Please generate hypotheses to support why the outcome of the case might be "+outcome+" in JSON form."
  return generate_thought(prompt)

#current_case = "The plaintiff steals the defendant's knife from the defendant in order to protect the plantiff's children from the defendant's knife attack"
current_case = "The plaintiff was fishing with a net in an open sea. While the net was almost closed, the defendant rowed his boat, entered the net, caught the fish, and rowed his boat out of the net with the fish. The plaintiff sued the defendant for theft."
rule = "A person is typically guilty of theft if the following conditions hold 1) a person committed an action to take away the thing. 2) the thing belonged to another person. 3) the person knew that the thing belonged to the other person. 4) the person had an intention to take away a thing. The rule is not applicable if the purpose of taking away is to prevent dangers."
precedent_1 = {"fact": "The plaintiff chased a fox in an open land. The defendant killed it and carried it away.", "outcome": "the defendant is not guilty of theft"}
precedent_2 = {"fact": "The plaintiff owned land containing a pond, where the plaintiff always takes ducks for a profit. The defendant, knowing about the decoy pond, fired the guns to scare away the ducks in the pond.", "outcome": "the defendant is guitly of theft"}
issue = "the defendant is guilty of theft"

print ("rule thought")
print(rule_thought(current_case,rule,issue))

# print ("case thought-1")
# new_rule1 = case_thought(precedent_1["fact"],precedent_1["outcome"])
# print(new_rule1)

#print ("case thought-2")
#new_rule2 = case_thought(precedent_2["fact"],precedent_2["outcome"])
#print(new_rule2)

# print ("rule-thought revised")
# print(rule_thought(current_case,rule+"\n"+new_rule1,issue))


# def solve_problem(problem):
#   # Define the GoT graph structure
#   graph = {
#     "node1": ["prompt1"],
#     "node2": ["prompt2", "thought1"],
#     "node3": ["prompt3", "thought2"]
#   }

#   # Generate thoughts
#   thoughts = {}
#   for node, prompts in graph.items():
#     thoughts[node] = [generate_thought(prompt) for prompt in prompts]

#   # Process and combine thoughts
#   # ...

#   # Return final result
#   return final_result


rule thought
Prompt:
Considering the following facts:
The plaintiff was fishing with a net in an open sea. While the net was almost closed, the defendant rowed his boat, entered the net, caught the fish, and rowed his boat out of the net with the fish. The plaintiff sued the defendant for theft.
Considering the following rules:
A person is typically guilty of theft if the following conditions hold 1) a person committed an action to take away the thing. 2) the thing belonged to another person. 3) the person knew that the thing belonged to the other person. 4) the person had an intention to take away a thing. The rule is not applicable if the purpose of taking away is to prevent dangers.
According to the given rules , please evaluate whether the defendant is guilty of theft.
Response:
Let's analyze the situation based on the provided rules:

**1. Action to take away the thing:** The defendant clearly took action by rowing his boat into the plaintiff's net, catching the fish, and rowing o

# Test 1: Rule-base only

In [ ]:
print ("rule thought")
print(rule_thought(current_case,rule,issue))

rule thought
Let's analyze the defendant's actions against the provided rules of theft:

**1. Action to take away the thing:** The defendant did indeed row his boat into the plaintiff's net, catch the fish, and row away. This constitutes a physical act of taking the fish.

**2. The thing belonged to another person:** The fish were caught in the plaintiff's net, indicating that the plaintiff had possession and ownership of the fish.

**3. Knowledge that the thing belonged to another person:** The defendant, by entering the net and taking the fish, clearly demonstrated awareness that the fish were in the plaintiff's possession. 

**4. Intention to take away a thing:** This is where it becomes more complex.  The rules do not specify what constitutes "intention".  It is unclear if the defendant's primary intention was to take the fish or if he was simply trying to maneuver his boat out of the net. 

**5.  Purpose of taking away is to prevent dangers:** The case does not mention any danger 

# Test 2: Hypo & Revised Rule-base

In [ ]:
print ("hypo thought")
hypo = case_thought(current_case,"the defendant is not guilty of theft")
print(hypo)

print ("revised rule thought")
print(rule_thought(current_case,rule,issue,hypo))

hypo thought
Prompt:
Considering the following case: 
The plaintiff was fishing with a net in an open sea. While the net was almost closed, the defendant rowed his boat, entered the net, caught the fish, and rowed his boat out of the net with the fish. The plaintiff sued the defendant for theft.
The case has the following outcome: 
the defendant is not guilty of theft
Please answer why the case has the outcome only in the following form: 
A person is typically guilty of theft if the following conditions hold: (1)... (2)... orA person is typically not guilty of theft if the following conditions hold: (1)... (2)...
Response:
A person is typically **not** guilty of theft if the following conditions hold: 
(1) The property in question is not considered "owned" by the plaintiff. 
(2) The plaintiff has not taken any actions to secure or possess the property. 

In this case, the fish were in the open sea, which is considered a common resource. The plaintiff had not yet fully enclosed the net,